In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import statsmodels.api as sm
import scipy.stats as stats

In [2]:
df= sm.datasets.fair.load_pandas().data
df

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666
...,...,...,...,...,...,...,...,...,...
6361,5.0,32.0,13.0,2.0,3.0,17.0,4.0,3.0,0.000000
6362,4.0,32.0,13.0,1.0,1.0,16.0,5.0,5.0,0.000000
6363,5.0,22.0,2.5,0.0,2.0,14.0,3.0,1.0,0.000000
6364,5.0,32.0,6.0,1.0,3.0,14.0,3.0,4.0,0.000000


In [3]:
 df['affair'] = (df.affairs > 0).astype(int)
for column in df.columns:
    df[column]=df[column].astype(int)

In [4]:
df

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs,affair
0,3,32,9,3,3,17,2,5,0,1
1,3,27,13,3,1,14,3,4,3,1
2,4,22,2,0,1,16,3,5,1,1
3,4,37,16,4,3,16,5,5,0,1
4,5,27,9,1,1,14,3,4,4,1
...,...,...,...,...,...,...,...,...,...,...
6361,5,32,13,2,3,17,4,3,0,0
6362,4,32,13,1,1,16,5,5,0,0
6363,5,22,2,0,2,14,3,1,0,0
6364,5,32,6,1,3,14,3,4,0,0


In [5]:
df.drop(columns=['affairs'],inplace=True)
df.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3,32,9,3,3,17,2,5,1
1,3,27,13,3,1,14,3,4,1
2,4,22,2,0,1,16,3,5,1
3,4,37,16,4,3,16,5,5,1
4,5,27,9,1,1,14,3,4,1


In [6]:
occup={1:'occu_1',2:'occu_2',3:'occu_3',4:'occu_4',5:'occu_5',6:'occu_6'}


In [7]:
df1=df.copy()
df1.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3,32,9,3,3,17,2,5,1
1,3,27,13,3,1,14,3,4,1
2,4,22,2,0,1,16,3,5,1
3,4,37,16,4,3,16,5,5,1
4,5,27,9,1,1,14,3,4,1


In [8]:
df1['occup']=df1['occupation'].map(occup)
df1['occup_husb']=df1['occupation_husb'].map(occup)
df1['affairs']=df1['affair'].apply(lambda x:"Yes" if x==1 else "No")
df1.head()


,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair,occup,occup_husb,affairs
0,3,32,9,3,3,17,2,5,1,occu_2,occu_5,Yes
1,3,27,13,3,1,14,3,4,1,occu_3,occu_4,Yes
2,4,22,2,0,1,16,3,5,1,occu_3,occu_5,Yes
3,4,37,16,4,3,16,5,5,1,occu_5,occu_5,Yes
4,5,27,9,1,1,14,3,4,1,occu_3,occu_4,Yes


In [9]:
df1.drop(columns=['occupation','occupation_husb','affair'],inplace=True)
df1.head()

,rate_marriage,age,yrs_married,children,religious,educ,occup,occup_husb,affairs
0,3,32,9,3,3,17,occu_2,occu_5,Yes
1,3,27,13,3,1,14,occu_3,occu_4,Yes
2,4,22,2,0,1,16,occu_3,occu_5,Yes
3,4,37,16,4,3,16,occu_5,occu_5,Yes
4,5,27,9,1,1,14,occu_3,occu_4,Yes


In [10]:
table=pd.crosstab(df1['occup'],df1['affairs']).values
table

array([[  34,    7],
       [ 607,  252],
       [1818,  965],
       [1354,  480],
       [ 431,  309],
       [  69,   40]], dtype=int64)

In [11]:
result=stats.chi2_contingency(table)[1]
result

2.477892436591016e-15

In [12]:
table=pd.crosstab(df1['occup_husb'],df1['affairs']).values
result=stats.chi2_contingency(table)[1]
result

0.0024564278771048156

In [13]:
df1.corr()

,rate_marriage,age,yrs_married,children,religious,educ
rate_marriage,1.000000,-0.111164,-0.130189,-0.131083,0.078794,0.079869
age,-0.111164,1.000000,0.892957,0.677503,0.136331,0.029066
yrs_married,-0.130189,0.892957,1.000000,0.778395,0.131568,-0.107169
children,-0.131083,0.677503,0.778395,1.000000,0.141760,-0.143294
religious,0.078794,0.136331,0.131568,0.141760,1.000000,0.032245
educ,0.079869,0.029066,-0.107169,-0.143294,0.032245,1.000000


In [14]:
df1.head()

,rate_marriage,age,yrs_married,children,religious,educ,occup,occup_husb,affairs
0,3,32,9,3,3,17,occu_2,occu_5,Yes
1,3,27,13,3,1,14,occu_3,occu_4,Yes
2,4,22,2,0,1,16,occu_3,occu_5,Yes
3,4,37,16,4,3,16,occu_5,occu_5,Yes
4,5,27,9,1,1,14,occu_3,occu_4,Yes


In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1
for column in df:
    if plotnumber<=df.shape[1]:
        ax=plt.subplot(3,3,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
        plotnumber+=1
        

In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1
for column in df:
    if plotnumber<=df.shape[1]:
        ax=plt.subplot(3,3,plotnumber)
        sns.countplot(df[column],hue=df.affair)
        plt.xlabel(column,fontsize=20)
        plotnumber+=1

In [ ]:
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  df, return_type="dataframe")


# rename column names for the dummy variables for better looks:
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})

y = np.ravel(y)

In [ ]:
X

In [ ]:
y

In [ ]:
df['affair'].value_counts()


In [ ]:
df.describe()

In [ ]:
df['yrs_married'].value_counts()


In [ ]:
df['affair'].value_counts()

In [ ]:
df.dtypes

In [ ]:
print(f"% of people not having affair is {df['affair'].value_counts()[0]*100/df.shape[0]}")
print(f"% of people having affair is {df['affair'].value_counts()[1]*100/df.shape[0]}")

In [ ]:

pd.crosstab(df.occupation,df.affair).plot(kind='bar')
plt.title("Affair on basis of occupation of wife")
plt.xlabel('occupation')
plt.ylabel("frequency")

In [ ]:
pd.crosstab(df.occupation_husb,df.affair).plot(kind='bar')
plt.title("Affair on basis of occupation of husband")
plt.xlabel('occupation')
plt.ylabel("frequency")

In [ ]:
pd.crosstab(df.rate_marriage,df.affair).plot(kind='bar')
plt.title("Affair on basis of rating")
plt.xlabel('occupation')
plt.ylabel("frequency")

In [ ]:

pd.crosstab(df.religious,df.affair).plot(kind='bar')
plt.title("Affair on basis of rating")
plt.xlabel('occupation')
plt.ylabel("frequency")

In [ ]:
pd.crosstab(df.age,df.affair).plot(kind='bar')
plt.title("Affair on basis of Age")
plt.xlabel('occupation')
plt.ylabel("frequency")

In [ ]:
X.drop(columns=['Intercept'],inplace=True)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [ ]:
X_scaled=sc.fit_transform(X.copy())


In [ ]:

vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"] = X.columns

#let's check the values
vif  

In [ ]:
results=sm.Logit(y,X).fit()
results.summary()

In [ ]:
X.drop(columns=['children'])

In [ ]:
X_scaled=sc.fit_transform(X.copy())
vif2 = pd.DataFrame()
vif2["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif2["Features"] = X.columns

#let's check the values
vif2  

In [ ]:
results=sm.Logit(y,X).fit()
results.summary()

In [ ]:
X.drop(columns=['children'],inplace=True)

In [ ]:
results=sm.Logit(y,X).fit()
results.summary()

In [ ]:
X.drop(columns=['educ'],inplace=True)

In [ ]:
results=sm.Logit(y,X).fit()
results.summary()

In [ ]:
X.drop(columns=['C(occupation_husb)[T.2]','C(occupation_husb)[T.3]','C(occupation_husb)[T.4]','C(occupation_husb)[T.5]','C(occupation_husb)[T.6]'],inplace=True)

In [ ]:

results=sm.Logit(y,X).fit()
results.summary()

In [ ]:
X_scaled=sc.fit_transform(X.copy())
vif2 = pd.DataFrame()
vif2["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif2["Features"] = X.columns

#let's check the values
vif2  

In [ ]:
X_scaled=sc.fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y, test_size= 0.25, random_state = 355)

In [ ]:
model = LogisticRegression()
model = model.fit(X_train, y_train)

model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
cm=confusion_matrix(y_test,model.predict(X_test))
cm

In [ ]:
cr=classification_report(y_test,model.predict(X_test))
print(cr)